In [1]:
import os
import random
import timeit
import pandas as pd
import numpy as np
import tempfile
import tensorflow as tf
import xgboost as xgb
import matplotlib.pyplot as plt
import tensorflow_model_optimization as tfmot
import keras.models as k_models

from scipy.sparse import csr_matrix, save_npz
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tf_keras as keras
from tf_keras import activations
from tf_keras.models import Model, Sequential, load_model
from tf_keras.layers import Dense, Input, LSTM
from tf_keras.callbacks import EarlyStopping


In [5]:
# MLP

In [6]:
# Load the model
mlp_model = k_models.load_model('MLP_HPO.keras')

In [7]:
df_A = pd.read_csv('Final_EVSE_A.csv')
df_B = pd.read_csv('Final_EVSE_B.csv')

def prepare_categorical_output(y):
    # # Print the `y` matrix before encoding
    # print("y matrix before encoding:\n", y)

    # Convert Series to NumPy array and reshape `y` matrix
    y = y.values.reshape(-1, 1)

    # Print the `y` matrix after reshaping
    # print("y matrix after reshaping:\n", y)
    
    # One-hot encode the target variable
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(y)

    # Print the `y` matrix after one-hot encoding
    # print("y matrix after one-hot encoding:\n", y)
    
    return y

#Considering B charging station as training and A as testing

def assigning_set(df1, df2):
    # Group by 'CSVNameFile' and split the last 20% of each group into the validation set
    train_list = []
    val_list = []

    grouped = df1.groupby('CSVNameFile')

    for _, group in grouped:
        split_index = int(len(group) * 0.8)
        train_list.append(group.iloc[:split_index])
        val_list.append(group.iloc[split_index:])

    # Concatenate the training and validation sets
    train_df = pd.concat(train_list).reset_index(drop=True)
    val_df = pd.concat(val_list).reset_index(drop=True)

    # Separate features and labels for train and validation sets
    X_train = train_df.drop(columns=['CSVNameFile', 'status', 'multiclass'])
    y_train = prepare_categorical_output(train_df['multiclass'])

    X_val = val_df.drop(columns=['CSVNameFile', 'status', 'multiclass'])
    y_val = prepare_categorical_output(val_df['multiclass'])

    # X_test and y_test from df2 remain unchanged for test evaluation
    X_test = df2.drop(columns=['CSVNameFile', 'status', 'multiclass'])
    y_test = prepare_categorical_output(df2['multiclass'])

    input_dim = X_train.shape[1]
    output_dim = len(np.unique(df1['multiclass']))

    return X_train, X_val, X_test, y_train, y_val, y_test, input_dim, output_dim

X_train, X_val, X_test, y_train, y_val, y_test, input_dim, output_dim = assigning_set(df_B, df_A)

In [11]:
mlp_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                      │ (None, 16)                  │             800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │           2,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,283 (133.92 KB)

 Trainable params: 11,427 (44.64 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,856 (89.29 KB)

In [12]:
# Recreating the model into compatibale sequential format, using the same architecture

# Get input and output shapes of the model
input_dim = mlp_model.input_shape[1]  # Get input dimension 
output_dim = mlp_model.output_shape[1]  # Get output dimension

print("Input Dimension:", input_dim)
print("Output Dimension:", output_dim)

# Recreate the model in Sequential format
new_mlp_model = Sequential([
    Input(shape=(input_dim,)), 
    Dense(16, activation='relu'),
    Dense(128, activation='relu'),  
    Dense(64, activation='relu'),   
    Dense(output_dim, activation='softmax')
])

# Compile the new model with the same settings
new_mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the new model
new_mlp_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                   callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
                   batch_size=32, epochs=50, verbose=1)


new_mlp_model.save('MLP.h5')

# Display the new model summary to confirm the architectur
new_mlp_model.summary()

Input Dimension: 49
Output Dimension: 3


Epoch 1/50


54921/54921 [==============================] - 52s 937us/step - loss: 1691.2175 - accuracy: 0.8793 - val_loss: 0.1635 - val_accuracy: 0.9336
Epoch 2/50
54921/54921 [==============================] - 52s 942us/step - loss: 12.0309 - accuracy: 0.9296 - val_loss: 0.1686 - val_accuracy: 0.9291
Epoch 3/50
54921/54921 [==============================] - 52s 942us/step - loss: 15.6137 - accuracy: 0.9268 - val_loss: 0.1677 - val_accuracy: 0.9343
Epoch 4/50
54921/54921 [==============================] - 52s 948us/step - loss: 5.1026 - accuracy: 0.9263 - val_loss: 0.1577 - val_accuracy: 0.9366
Epoch 5/50
54921/54921 [==============================] - 52s 945us/step - loss: 9.3021 - accuracy: 0.9299 - val_loss: 0.1781 - val_accuracy: 0.9319
Epoch 6/50
54921/54921 [==============================] - 53s 956us/step - loss: 0.1826 - accuracy: 0.9303 - val_loss: 0.2227 - val_accuracy: 0.9165
Epoch 7/50
54921/54921 [==============================] -

C:\Users\mnoorche\AppData\Local\anaconda3\Lib\site-packages\tf_keras\src\engine\training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [186]:
y_pred = new_mlp_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')
print(f'Test F1 Score: {f1:.4f}')


17121/17121 [==============================] - 14s 841us/step
Test Accuracy: 0.9841
Test Precision: 0.9858
Test Recall: 0.9841
Test F1 Score: 0.9846


In [188]:
# Function to save weights to CSV
def save_weights_to_csv(model, file_prefix):
    for layer in model.layers:
        weights = layer.get_weights()  # This returns a list of NumPy arrays (weights and biases)
        for i, weight in enumerate(weights):
            layer_name = layer.name
            weight_type = 'weights' if i == 0 else 'biases'  # Name based on weights or biases
            file_name = f"{file_prefix}_{layer_name}_{weight_type}.csv"
            
            # Save the weights or biases to CSV
            pd.DataFrame(weight).to_csv(file_name, index=False)
            print(f"Saved {weight_type} for layer {layer_name} to {file_name}")

# Save weights for the original model
save_weights_to_csv(new_mlp_model, "before_pruning_MLP")

Saved weights for layer dense to before_pruning_MLP_dense_weights.csv
Saved biases for layer dense to before_pruning_MLP_dense_biases.csv
Saved weights for layer dense_1 to before_pruning_MLP_dense_1_weights.csv
Saved biases for layer dense_1 to before_pruning_MLP_dense_1_biases.csv
Saved weights for layer dense_2 to before_pruning_MLP_dense_2_weights.csv
Saved biases for layer dense_2 to before_pruning_MLP_dense_2_biases.csv
Saved weights for layer dense_3 to before_pruning_MLP_dense_3_weights.csv
Saved biases for layer dense_3 to before_pruning_MLP_dense_3_biases.csv


In [196]:
def pruning(model, X_train, X_val, X_test, y_train, y_val, y_test, final_sparsity, df_name):
    
    # dataset_size = the number of samples in the training set.
    # batch_size = the number of samples processed in one training step.
    # num_epochs = the number of times the entire training set is used to update the model.
    
    # Parameters for the dataset and training
    dataset_size = len(X_train)
    batch_size = 32  
    num_epochs = 20
    
    # Pruning parameters as percentages of the total steps
    start_pct = 0  # Start pruning at the beginning of the training steps
    end_pct = 0.4    # End pruning after 40% of the training steps

    # Step calculations
    steps_per_epoch = dataset_size / batch_size
    total_steps = steps_per_epoch * num_epochs

    start_step = int(total_steps * start_pct)
    end_step = int(total_steps * end_pct)

    # Display the calculated steps
    print(f"Total Steps: {total_steps}")
    print(f"Start Step: {start_step}")
    print(f"End Step: {end_step}")
    
    start_epoch = start_step // steps_per_epoch
    end_epoch = end_step // steps_per_epoch
    print(f"Pruning will start in epoch {int(start_epoch)} and end in epoch {int(end_epoch)}")

    
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.60,
                                                                 final_sparsity=final_sparsity,
                                                                 begin_step=start_step,
                                                                 end_step=end_step,
                                                                 power=1,
                                                                 frequency=100),
    }

    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(
        model, **pruning_params
    )   

    
    # Compile the pruned model
    pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


    # Set up pruning callbacks and early stopping
    pruning_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=tempfile.mkdtemp())
    ]

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
      
    callbacks = pruning_callbacks + [early_stopping]

    history = pruned_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=num_epochs, batch_size=batch_size, callbacks=callbacks)

    # Strip pruning wrappers
    final_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

    # Compile the model again after stripping the pruning wrappers
    final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    final_model.save(f'{df_name}.h5') 
    
    # The number of epochs should be greater than num_epochs_for_end_step to make sure pruning is complete
    num_epochs_for_end_step = int(end_step // steps_per_epoch)
    the_number_of_epochs = len(history.history['loss'])
    # Print the number of epochs and the epoch when pruning finished
    print(f"Number of epochs is: {the_number_of_epochs}, and pruning finished at epoch {num_epochs_for_end_step}")
    
    # Check if pruning completed before early stopping
    pruning_completed = the_number_of_epochs > num_epochs_for_end_step
    print(f"Pruning completed before early stopping: {pruning_completed}")

    y_pred = final_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test Precision: {precision:.4f}')
    print(f'Test Recall: {recall:.4f}')
    print(f'Test F1 Score: {f1:.4f}')

    # Save weights for the pruned model
    save_weights_to_csv(final_model, f'after_pruning_{df_name}')
    
    return final_model, pruning_completed

In [198]:
the_mlp_model = load_model('MLP.h5')
PrunedMLP, pruning_completed = pruning(the_mlp_model, X_train, X_val, X_test, y_train, y_val, y_test, 0.65, 'PrunedMLP')

Total Steps: 1098415.0
Start Step: 0
End Step: 439366
Pruning will start in epoch 0 and end in epoch 8
Epoch 1/20
54921/54921 [==============================] - 96s 2ms/step - loss: 197.1284 - accuracy: 0.9176 - val_loss: 0.1479 - val_accuracy: 0.9163
Epoch 2/20
54921/54921 [==============================] - 93s 2ms/step - loss: 1.6267 - accuracy: 0.9313 - val_loss: 0.1874 - val_accuracy: 0.9080
Epoch 3/20
54921/54921 [==============================] - 93s 2ms/step - loss: 639.7868 - accuracy: 0.9274 - val_loss: 0.1722 - val_accuracy: 0.9276
Epoch 4/20
54921/54921 [==============================] - 93s 2ms/step - loss: 6.6030 - accuracy: 0.9314 - val_loss: 0.1543 - val_accuracy: 0.9364
Epoch 5/20
54921/54921 [==============================] - 93s 2ms/step - loss: 6.4820 - accuracy: 0.9343 - val_loss: 0.1434 - val_accuracy: 0.9362
Epoch 6/20
54921/54921 [==============================] - 93s 2ms/step - loss: 113.2824 - accuracy: 0.9346 - val_loss: 0.1435 - val_accuracy: 0.9364
Epoch 7/2

C:\Users\mnoorche\AppData\Local\anaconda3\Lib\site-packages\tf_keras\src\engine\training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


17121/17121 [==============================] - 14s 837us/step
Test Accuracy: 0.9804
Test Precision: 0.9854
Test Recall: 0.9804
Test F1 Score: 0.9817
Saved weights for layer dense to after_pruning_PrunedMLP_dense_weights.csv
Saved biases for layer dense to after_pruning_PrunedMLP_dense_biases.csv
Saved weights for layer dense_1 to after_pruning_PrunedMLP_dense_1_weights.csv
Saved biases for layer dense_1 to after_pruning_PrunedMLP_dense_1_biases.csv
Saved weights for layer dense_2 to after_pruning_PrunedMLP_dense_2_weights.csv
Saved biases for layer dense_2 to after_pruning_PrunedMLP_dense_2_biases.csv
Saved weights for layer dense_3 to after_pruning_PrunedMLP_dense_3_weights.csv
Saved biases for layer dense_3 to after_pruning_PrunedMLP_dense_3_biases.csv


In [202]:
# Function to compare weights before and after pruning, and calculate percentage of zero weights
def compare_weights_before_after(model_before, model_after):
    for layer in model_before.layers:
        # Get the layer name
        layer_name = layer.name
        
        # Get weights and biases from both models
        weights_before = model_before.get_layer(layer_name).get_weights()
        weights_after = model_after.get_layer(layer_name).get_weights()
        
        print(f"Comparing weights for layer: {layer_name}")
        
        for i, (before, after) in enumerate(zip(weights_before, weights_after)):
            weight_type = 'weights' if i == 0 else 'biases'
            
            # Calculate the total number of weights
            total_weights = np.prod(before.shape)
            
            # Calculate the number of weights that have changed to zero
            changes_to_zero = (before != 0) & (after == 0)
            num_changes_to_zero = np.sum(changes_to_zero)
            
            # Calculate the number of zero weights after pruning
            num_zero_weights_after = np.sum(after == 0)
            
            # Calculate the percentage of weights that are now zero
            percentage_zero_weights = (num_zero_weights_after / total_weights) * 100
            
            # Print the results
            print(f"{weight_type.capitalize()} in layer '{layer_name}':")
            print(f"  Total number of weights: {total_weights}")
            print(f"  Weights changed to zero during pruning: {num_changes_to_zero}")
            print(f"  Percentage of zero weights after pruning: {percentage_zero_weights:.2f}%")
            print("")

# Call the function to compare the original model and the pruned model
compare_weights_before_after(new_mlp_model, PrunedMLP)


Comparing weights for layer: dense
Weights in layer 'dense':
  Total number of weights: 784
  Weights changed to zero during pruning: 510
  Percentage of zero weights after pruning: 65.05%

Biases in layer 'dense':
  Total number of weights: 16
  Weights changed to zero during pruning: 0
  Percentage of zero weights after pruning: 0.00%

Comparing weights for layer: dense_1
Weights in layer 'dense_1':
  Total number of weights: 2048
  Weights changed to zero during pruning: 1331
  Percentage of zero weights after pruning: 64.99%

Biases in layer 'dense_1':
  Total number of weights: 128
  Weights changed to zero during pruning: 0
  Percentage of zero weights after pruning: 0.00%

Comparing weights for layer: dense_2
Weights in layer 'dense_2':
  Total number of weights: 8192
  Weights changed to zero during pruning: 5325
  Percentage of zero weights after pruning: 65.00%

Biases in layer 'dense_2':
  Total number of weights: 64
  Weights changed to zero during pruning: 0
  Percentage o

In [204]:
# Reducing size

In [206]:
for layer in PrunedMLP.layers:
    weights = layer.get_weights()
    if len(weights) > 0:
        print(f"Layer: {layer.name}, Weight dtype: {weights[0].dtype}, Bias dtype: {weights[1].dtype}" if len(weights) > 1 else f"Layer: {layer.name}, Weight dtype: {weights[0].dtype}")

Layer: dense, Weight dtype: float32, Bias dtype: float32
Layer: dense_1, Weight dtype: float32, Bias dtype: float32
Layer: dense_2, Weight dtype: float32, Bias dtype: float32
Layer: dense_3, Weight dtype: float32, Bias dtype: float32


In [208]:
# Function to perform inference with sparse matrix multiplication while keeping everything sparse
def sparse_inference(model, input_data):
    layer_outputs = input_data  # Start with input data (dense)

    # start_time = timeit.default_timer()  # Start timing the inference

    for layer in model.layers:
        if 'dense' in layer.name:  # Only focus on Dense layers
            dense_weights = layer.get_weights()[0]  # Get dense weights
            bias = layer.get_weights()[1]  # Get biases

            # print(f"Layer: {layer.name}, Weight shape: {dense_weights.shape}, Bias shape: {bias.shape}")
            # print(f"Input shape before multiplication: {layer_outputs.shape}")

            # Convert dense weights to sparse matrix
            sparse_weights = csr_matrix(dense_weights)

            # Perform matrix multiplication
            layer_outputs = layer_outputs @ sparse_weights  # Matrix multiplication
            # print(f"Output shape after multiplication: {layer_outputs.shape}")

            # Add bias to each output row
            layer_outputs = layer_outputs + bias
            # print(f"Output shape after adding bias: {layer_outputs.shape}")

            # Apply the activation function to the non-zero values in the sparse matrix
            if layer.activation is not None:
                activation_function = layer.activation  # Get the activation function
                
                # Manually apply relu or softmax to sparse matrix
                if activation_function == activations.relu:
                    layer_outputs.data = np.maximum(0, layer_outputs.data)  # Apply ReLU to non-zero values

                elif activation_function == activations.softmax:
                    # Apply softmax directly on 1D output (we are at the final output layer)
                    if layer_outputs.shape[1] == 3:  # If we are at the final layer with 3 outputs
                        exp_values = np.exp(layer_outputs - np.max(layer_outputs))
                        softmax_outputs = exp_values / np.sum(exp_values, axis=1)
                        layer_outputs = softmax_outputs
                        # print(f"Softmax applied: {softmax_outputs}")
                else:
                    # For other activations, raise an error
                    raise NotImplementedError(f"Activation {activation_function} is not implemented for sparse matrices")

                # print(f"Output shape after applying activation to non-zero values: {layer_outputs.shape}")

    # end_time = timeit.default_timer()  # Stop timing the inference

    # inference_time = end_time - start_time  # Calculate total inference time
    # print(f"Total inference time: {inference_time * 1000:.6f} milliseconds")  # Print time in milliseconds

    return layer_outputs

In [210]:
# Example input for inference
input_data = np.random.rand(1, 49) 


# Perform inference using sparse matrix multiplication
sparse_inference_output = sparse_inference(PrunedMLP, input_data)
print('Prediction of the PrunedMLP using sparse_inference function:', sparse_inference_output)

predict_pruned_inference_output = PrunedMLP.predict(input_data)
print('Prediction of the PrunedMLP using predict method:', predict_pruned_inference_output)

predict_original_inference_output = new_mlp_model.predict(input_data)
print('Prediction of the original MLP using predict method:', predict_original_inference_output)

C:\Users\mnoorche\AppData\Local\Temp\ipykernel_1244\2912187159.py:32: DeprecationWarning: Assigning the 'data' attribute is an inherently unsafe operation and will be removed in the future.
  layer_outputs.data = np.maximum(0, layer_outputs.data)  # Apply ReLU to non-zero values


Prediction of the PrunedMLP using sparse_inference function: [[3.04889482e-006 1.97502139e-119 9.99996951e-001]]
1/1 [==============================] - 1s 569ms/step
Prediction of the PrunedMLP using predict method: [[3.0488914e-06 0.0000000e+00 9.9999690e-01]]
1/1 [==============================] - 0s 21ms/step
Prediction of the original MLP using predict method: [[8.812600e-03 1.119682e-12 9.911874e-01]]


In [212]:
def measure_inference_time(model, pruned_model, X_test):
    # Lists to store inference times for both models
    model_time_list = []
    pruned_model_time_list = []

    # Model before pruning (original model)
    for sample in X_test:
        start_time = timeit.default_timer()
        model.predict(np.expand_dims(sample, axis=0), verbose=False)
        end_time = timeit.default_timer()
        model_time_list.append((end_time - start_time) * 1000)  # Convert to milliseconds
    
    avg_inference_time_model = np.mean(model_time_list)
    print(f"Average inference time per sample for Model before pruning: {avg_inference_time_model:.6f} milliseconds")

    # Model after pruning (pruned model)
    for sample in X_test:
        start_time = timeit.default_timer()
        sparse_inference(pruned_model, np.expand_dims(sample, axis=0))
        end_time = timeit.default_timer()
        pruned_model_time_list.append((end_time - start_time) * 1000)  # Convert to milliseconds
    
    avg_inference_time_pruned_model = np.mean(pruned_model_time_list)
    print(f"Average inference time per sample for Model after pruning: {avg_inference_time_pruned_model:.6f} milliseconds")
   
    # Save the lists as NumPy arrays
    np.save('model_time_list.npy', np.array(model_time_list))
    np.save('pruned_model_time_list.npy', np.array(pruned_model_time_list))
    
    return avg_inference_time_model, avg_inference_time_pruned_model


In [216]:
X_test.shape

(547854, 49)

In [226]:
X_test_subsample = X_test.sample(frac=0.1, random_state=42)

print(f"Subsampled X_test shape: {X_test_subsample.shape}")


Subsampled X_test shape: (54785, 49)


In [228]:
# Convert DataFrame to NumPy array
X_test_subsample_numpy = X_test_subsample.to_numpy(dtype=np.float32)
avg_time_model, avg_time_pruned_model = measure_inference_time(new_mlp_model, PrunedMLP, X_test_subsample_numpy)

Average inference time per sample for Model before pruning: 63.704041 milliseconds


C:\Users\mnoorche\AppData\Local\Temp\ipykernel_1244\2912187159.py:32: DeprecationWarning: Assigning the 'data' attribute is an inherently unsafe operation and will be removed in the future.
  layer_outputs.data = np.maximum(0, layer_outputs.data)  # Apply ReLU to non-zero values


Average inference time per sample for Model after pruning: 3.016179 milliseconds


In [7]:
# Function to save all weights of the original model in a single .npz file
def save_original_weights_as_npz(model, file_name):
    all_weights = {}
    for i, layer in enumerate(model.layers):
        weights = layer.get_weights()
        if weights:
            all_weights[f"layer_{i}_weights"] = weights[0]  # Save weights
            all_weights[f"layer_{i}_biases"] = weights[1]  # Save biases

    np.savez(file_name, **all_weights)
    print(f"Saved original model weights to {file_name}.npz")

# Function to save all pruned (sparse) weights in a single .npz file
def save_pruned_weights_as_npz(model, file_name):
    all_sparse_weights = {}
    for i, layer in enumerate(model.layers):
        weights = layer.get_weights()
        if weights:
            sparse_weight = csr_matrix(weights[0])  # Convert dense weights to sparse
            all_sparse_weights[f"layer_{i}_sparse_weights"] = sparse_weight
            all_sparse_weights[f"layer_{i}_biases"] = weights[1]  # Save biases as normal

    # Save the sparse weights in .npz format
    np.savez(file_name, **all_sparse_weights)
    print(f"Saved pruned model weights to {file_name}.npz")

# Save all weights for original and pruned models
save_original_weights_as_npz(new_mlp_model, 'original_model_weights')
save_pruned_weights_as_npz(PrunedMLP, 'pruned_model_weights')


# Function to compare the sizes of two files
def compare_file_sizes(file1, file2):
    size1 = os.path.getsize(file1) / 1024  # Convert to KB
    size2 = os.path.getsize(file2) / 1024  # Convert to KB
    print(f"Size of {file1}: {size1:.2f} KB")
    print(f"Size of {file2}: {size2:.2f} KB")

# Comparing the size of the original and pruned model weight files
compare_file_sizes('original_model_weights.npz', 'pruned_model_weights.npz')

Saved original model weights to original_model_weights.npz
Saved pruned model weights to pruned_model_weights.npz
Size of original_model_weights.npz: 46.70 KB
Size of pruned_model_weights.npz: 36.46 KB


In [232]:
# # Loading the weights from a saved .npz file
# def load_weights_from_npz(file_name):
#     loaded = np.load(file_name)
#     print(f"Loaded weights from {file_name}")
#     return loaded

# # Load the saved weights for both models
# original_weights = load_weights_from_npz('original_model_weights.npz')
# pruned_weights = load_weights_from_npz('pruned_model_weights.npz')

In [9]:
# new_mlp_model = load_model('MLP.h5')
# PrunedMLP = load_model('PrunedMLP.h5')